<img src='https://upload.wikimedia.org/wikipedia/en/0/0d/American_beauty_soundtrack.jpg' width=50%, height=auto>

# Recommendation Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 결과만 보기
___

### Top rating #30

In [118]:
top_rating_movies

title
american beauty                                   3211
star wars: episode iv - a new hope                2910
star wars: episode v - the empire strikes back    2885
star wars: episode vi - return of the jedi        2716
saving private ryan                               2561
terminator 2: judgment day                        2509
silence of the lambs, the                         2498
raiders of the lost ark                           2473
back to the future                                2460
matrix, the                                       2434
jurassic park                                     2413
sixth sense, the                                  2385
fargo                                             2371
braveheart                                        2314
men in black                                      2297
schindler's list                                  2257
princess bride, the                               2252
shakespeare in love                               2213
l.a.

### Configuration of Model Parameters

In [119]:
# 조금 걸렸습니다.
dict_

{'dtype': numpy.float32,
 'factors': 1000,
 'iterations': 30000,
 'regularization': 0.0001,
 'use_gpu': True}

### Rating of My Favorite predited by Model

In [120]:
# user와 content의 행렬 내적
# 내가 좋아하는 영화 Titanic을 모델이 얼마나 좋아하나 예측한 값
# 벡터 내적값
f"{np.dot(my_vector_, titanic_vector_)*100:.3f}%"

'79.984%'

### Recommend Movie from Content seen

In [132]:
# 컨텐츠를 기반으로 추천
ls_recem_movie

['simon sez',
 'gay deceivers, the',
 'kama sutra: a tale of love',
 'associate, the']

## Path to my Drive

In [50]:
path = "/content/drive/MyDrive/Ex/Ex-15"

## Index
___



1.   항목 추가

1.   항목 추가

1.   항목 추가
2.   항목 추가


2.   항목 추가


2.   항목 추가



## Set up

### Pip
___

In [51]:
%%capture
# need to install in colab
!pip install implicit

### Import Module
___

In [52]:
import os
import sys
import pickle
import random

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

import tensorflow as tf

In [53]:
from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares

### Configuration

In [54]:
random_seed = 1234

random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

In [55]:
%matplotlib inline

plt.style.use('seaborn')
mpl.rcParams['figure.figsize'] = (10, 10)
# mpl.rcParams['xtick.labelbottom'] = False
# mpl.rcParams['ytick.labelleft'] = False
mpl.rcParams['axes.grid'] = False
mpl.rcParams.update({'figure.autolayout': True})

warnings.filterwarnings('ignore')

In [56]:
# configuration recommended by implicit library
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [57]:
import numpy as np
import scipy
import implicit

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.6
1.4.1
0.5.2


## Preface
___

### Recommender system

_What is recommendation system?_
> 나와 성향이 비슷한 사람들이 좋아하는 것을 나에게 추천한다.

There are __two parts__ of recommendation algorithm.

1. Collarative Filtering
2. Content-based Filtering

### 1. Collarative Filtering
_협업 필터링_
___

<font size=4><b>기존 사용자의 행동 패턴을 분석하여 비슷한 이용자에게 기존 사용자의 패턴대로 추천하는 것.  
대표적인 예로 '이 상품을 구매한 사용자가 같이 본 상품'을 들 수 있다.  </b></font>
<span style="line-height:0.5em">
</br>  
대표 알고리즘 
 - Matrix Factorization  
 - k-nearest neighbor algorithm; kNN

<p>

단점 
 - Cold Start : 신곡과 같은 분석되지 않은 정보로는 추천할 수 없다. 
 - Long tail : 비대칭적으로 쏠린 정보에 의해 사용자들에게 관심이 없는 자료는 분석하기 어려워 충분한 정보를 제공할 수 없다.


### 2. Content-based Filtering
_콘텐츠 기반 필터링_
___

<font size=4> __사용자가 아닌 컨텐츠 자체를 분석하는 것으로 비슷한 결의 콘텐츠를 추천하는 것__  
item profile(항목 분석)과 user profile(사용자의 선호도)의 유사성을 측정한 뒤 사용자가 좋아할 만한 컨텐츠를 제공한다.  </font>  
<span style="line-height:0.5em">
</br>
대표 알고리즘 
 - Clustrering analysis(군집 분석)  
 - Artificial neural network
 - tf-dif(term frequency-inverse document frequency)

### Social Issue
___

___Filter Bubble___

추천 시스템이 발전함에 따라서 사용자의 기존 성향대로 편향되게 정보에 노출되는 현상  
일반 대중들은 경각심을 가질 필요가 있다고 생각한다.  


### Rating Matrix
_평가 행렬_  
_row : user_  
_column : content_   
_element : rating_ 

#### Matrix Factorization
_MF, 행렬분해_  
_Collabarative Filtering_

</br>
<img src='https://velog.velcdn.com/images/jujemu/post/368a6c4e-e050-4c01-bd82-c8ea4dd1c33e/image.png' width=80% height=auto></img>

Rating Matrix; ___R___ 을 matrix ___P___,___Q___의 곱으로 표현  
각 ___P___와 ___Q___는 ___k___ 차원을 갖는 Feature matrix  
___P___ : user  
___Q___ : content  

<font size=5> _Example_ </font>

<img src='https://velog.velcdn.com/images/jujemu/post/3d3d9d95-9734-4d9d-94ba-24533e8ec72a/image.png'></img>

_그림 출처 : CuMF_SGD: Fast and Scalable Matrix Factorization(https://arxiv.org/pdf/1610.05838.pdf)_


#### Compressed Sparse Row Matrix
_CSR_
_Sparse Matrix_

Rating Matrix가 가진 문제점은 모든 user와 모든 content의 정보가 담긴 행렬이라는 점이다.  

항의 대부분이 유의미하지 않은 값으로 채워진 이러한 행렬을 ___Sparse Matrx___이라고 한다.

_How to make CSR Matrix : https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html_



### <font size=5 color='blue'>Today's targeting model
___

행렬분해 모델에서 user, content feature matrix를 적절한 k 차원에서 행렬곱을 실행하여 얻은 행렬에서,  
예측된 유저의 컨텐츠 선호도가 실제 유저의 컨텐츠 평가 수치에 가깝게 만드는 것

# 참고
___

- 컨텐츠 알고리즘의 진화  
http://dl.moazine.com/viewer/index.asp?libraryid=9MtJb2T3nzH3yk7c212iu821DPFsY2ECOso0&a_i=8wwot4affaz3r6syx11VOYm2&keyword=&s_i=9#page/4

- Collaborative Filtering for Implicit Feedback Datasets  
http://yifanhu.net/PUB/cf.pdf

- kakao arena; 한국 멜론 음악 관련 정보  
https://arena.kakao.com/c/7

# 실습

In [58]:
rating_file_path = path + '/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [59]:
# 필요없는 항 timestamp 삭제
del ratings['timestamp']
ratings.head()

,user_id,movie_id,ratings
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [60]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [61]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [62]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
# 메타 데이터란, 다른 데이터의 정보가 들어있는 데이터
movie_file_path = path + '/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [63]:
ratings.columns, movies.columns

(Index(['user_id', 'movie_id', 'counts'], dtype='object'),
 Index(['movie_id', 'title', 'genre'], dtype='object'))

In [64]:
len_original = len(ratings)
len(ratings), len(movies)

(836478, 3883)

In [65]:
# ratings에 있는 영화가 movies에 모두 있는지 확인
# id의 최대값 확인 --> 동일
ratings['movie_id'].max(), movies['movie_id'].max()

(3952, 3952)

In [66]:
# 비교를 위해서 정렬 후 numpy로 저장
m = np.array(movies['movie_id'].sort_values())
m

array([   1,    2,    3, ..., 3950, 3951, 3952])

In [67]:
r = np.array(ratings['movie_id'].sort_values().unique())
r

array([   1,    2,    3, ..., 3950, 3951, 3952])

In [68]:
# 다음의 값으로 완전히 동일하지 않은 것으로 보이며
# ratings movie_id가 movies에 존재하는 것만 확인하면 된다.
len(m), len(r)

(3883, 3628)

In [69]:
# ratings의 영화가 movies에 모두 있는 것을 확인
mask = ratings['movie_id'].isin(m)
mask[mask == False].sum()

0

In [70]:
id_to_movie = {k:v for k,v in zip(movies['movie_id'], movies['title'])}

In [71]:
ratings['title'] = ratings['movie_id'].map(id_to_movie.get)

In [72]:
ls = ratings.columns.to_list()
ls

['user_id', 'movie_id', 'counts', 'title']

In [73]:
ls = ['title', 'counts', 'user_id','movie_id']
ratings = ratings[ls]
ratings.head()

,title,counts,user_id,movie_id
0,One Flew Over the Cuckoo's Nest (1975),5,1,1193
1,James and the Giant Peach (1996),3,1,661
2,My Fair Lady (1964),3,1,914
3,Erin Brockovich (2000),4,1,3408
4,"Bug's Life, A (1998)",5,1,2355


In [74]:
# 제대로 title 데이터 삽입이 됐는지 확인합니다.
movies[movies['movie_id'] == 661]

,movie_id,title,genre
655,661,James and the Giant Peach (1996),Animation|Children's|Musical


In [75]:
# title 컬럼에 결측값이 있는지 확인
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836478 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   title     836478 non-null  object
 1   counts    836478 non-null  int64 
 2   user_id   836478 non-null  int64 
 3   movie_id  836478 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 31.9+ MB


In [76]:
# Indexing
# 이미 id로 index되어있지만
# 비어있는 index가 있으므로 다시 설정한다.
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [77]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # ratings['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,title,counts,user_id,movie_id
0,One Flew Over the Cuckoo's Nest (1975),5,0,0
1,James and the Giant Peach (1996),3,0,1
2,My Fair Lady (1964),3,0,2
3,Erin Brockovich (2000),4,0,3
4,"Bug's Life, A (1998)",5,0,4
...,...,...,...,...
1000203,Platoon (1986),3,6038,1030
1000205,"Crying Game, The (1992)",5,6038,986
1000206,Welcome to the Dollhouse (1995),5,6038,311
1000207,Sophie's Choice (1982),4,6038,142


In [78]:
# 연도를 제외하고 영화 제목을 소문자화하여
# 검색하기 쉽도록 합니다.
def title_(ele):
    if ele[-6] == '(':
        ele = ele[:-6]
    return str.lower(ele).strip()

ratings['title'] = ratings['title'].map(title_)
ratings.head()

,title,counts,user_id,movie_id
0,one flew over the cuckoo's nest,5,0,0
1,james and the giant peach,3,0,1
2,my fair lady,3,0,2
3,erin brockovich,4,0,3
4,"bug's life, a",5,0,4


In [79]:
# id <--> movie 딕셔너리를 재정의합니다.
a = ratings['movie_id']
b = ratings['title']

id_to_movie = {k:v for k,v in zip(a, b)}
movie_to_id = {v:k for k,v in id_to_movie.items()}

In [80]:
top_rating_movies = ratings.groupby('title')['counts'].count().sort_values(ascending=False).head(30)
top_rating_movies.columns = 'counts'
top_rating_movies

title
american beauty                                   3211
star wars: episode iv - a new hope                2910
star wars: episode v - the empire strikes back    2885
star wars: episode vi - return of the jedi        2716
saving private ryan                               2561
terminator 2: judgment day                        2509
silence of the lambs, the                         2498
raiders of the lost ark                           2473
back to the future                                2460
matrix, the                                       2434
jurassic park                                     2413
sixth sense, the                                  2385
fargo                                             2371
braveheart                                        2314
men in black                                      2297
schindler's list                                  2257
princess bride, the                               2252
shakespeare in love                               2213
l.a.

In [81]:
# 좋아하는 영화를 추가한 뒤 movies 데이터 프레임에 존재하는지 확인합니다.
my_favorite = ['godfather, the', 'men in black', 'forrest gump', 'titanic', 'toy story', 'matrix, the']
for my in my_favorite:
  print(my in ratings['title'].to_list())
  print(movie_to_id[my])
  print()

True
607

True
175

True
160

True
1626

True
40

True
124



In [82]:
ls_id =[607, 175, 160, 1626, 40, 124]

In [83]:
# 저의 id를 max값 다음으로 넣겠습니다.
ratings['user_id'].nunique(), ratings['user_id'].max()

(6039, 6038)

In [84]:
# ratings['user_id'].max() is 6038
my_list = pd.DataFrame({'user_id': [6039]*6, 'title': my_favorite, 'counts':[5]*6, 'movie_id' : ls_id})
ratings = ratings.append(my_list)
ratings.tail(7)

,title,counts,user_id,movie_id
1000208,e.t. the extra-terrestrial,4,6038,26
0,"godfather, the",5,6039,607
1,men in black,5,6039,175
2,forrest gump,5,6039,160
3,titanic,5,6039,1626
4,toy story,5,6039,40
5,"matrix, the",5,6039,124


In [85]:
# 결측치가 존재하는지 확인
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836484 entries, 0 to 5
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   title     836484 non-null  object
 1   counts    836484 non-null  int64 
 2   user_id   836484 non-null  int64 
 3   movie_id  836484 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 31.9+ MB


### Modeling
___

In [86]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836484 stored elements in Compressed Sparse Row format>

In [87]:
# Matrix row x column 확인
num_user, num_movie

(6040, 3628)

In [111]:
# Configuration of Model
dict_ = dict(factors=1000, regularization=0.0001, use_gpu=True, iterations=30000, dtype=np.float32)
dict_

{'dtype': numpy.float32,
 'factors': 1000,
 'iterations': 30000,
 'regularization': 0.0001,
 'use_gpu': True}

In [112]:
als_model = AlternatingLeastSquares(**dict_)

In [113]:
# aiffel cloud환경에서 implicit version : 0.4.8
# colab 환경에서 implicit version : 0.5.2
# csr_data의 transpose는 aiffel환경에서만 필요한 것같다.

als_model.fit(csr_data)

  0%|          | 0/30000 [00:00<?, ?it/s]

In [91]:
# 모델이 history를 반환하는 것을 확인해보았더니
# 아무것도 반환하지 않고 활용할 수 있는것도 없네요
als_model?

In [114]:
# 유저는 처음부터 index되어 있었기에
# length - 1 값으로 대입
titanic = movie_to_id['titanic']
my_vector, titanic_vector = als_model.user_factors[6039], als_model.item_factors[titanic]

In [115]:
my_vector

Matrix([[-5.49751939e-03  3.31124291e-02 -5.27904183e-02 -5.91633506e-02
   9.38164890e-02 -5.72143793e-02 -1.09355450e-02  3.36916745e-02
  -4.63183075e-02  1.18602231e-01  5.03438041e-02 -6.84403162e-03
   1.66688591e-01 -7.07838759e-02 -2.00761873e-02  9.51767936e-02
  -3.38523202e-02 -5.99106625e-02 -7.89882615e-02 -5.45236208e-02
   4.59660478e-02  3.74794304e-02 -8.31061751e-02 -1.12100832e-01
  -3.68615948e-02  9.87141281e-02  6.97599575e-02 -1.05432823e-01
  -4.01524454e-02 -1.54032841e-01  1.09464996e-01 -1.68287102e-03
   6.12985454e-02 -1.93929002e-02 -3.00223976e-02 -5.48625514e-02
   8.64231139e-02  9.39319804e-02  9.03220568e-03  4.42842618e-02
   4.67040874e-02  1.39478575e-02  1.16653681e-01 -1.88150898e-01
   1.06934793e-01  6.96910098e-02 -2.93118004e-02 -6.77747801e-02
  -9.04228315e-02  9.93331298e-02  4.24652807e-02 -3.24313045e-02
  -1.26462933e-02  4.77715349e-03 -1.55816751e-03 -3.85232270e-02
   1.01008072e-01 -4.50075045e-02 -2.39725951e-02 -5.60297072e-02
  -

In [116]:
titanic_vector

Matrix([[-3.08644236e-03 -1.26547236e-02 -4.87071313e-02 -5.95807545e-02
   9.04280692e-02 -2.45258342e-02  6.86027016e-03  1.56147219e-03
  -8.17718823e-03  4.42911647e-02  4.14472632e-02 -2.27536145e-03
  -4.75000357e-03  4.09110449e-03 -1.14378510e-02  4.20534015e-02
   2.66308547e-03  2.13193856e-02 -3.00285909e-02 -2.11802926e-02
  -3.67777720e-02 -1.69554986e-02 -4.54403386e-02 -1.92715302e-02
   5.13796834e-03  2.38904655e-02  3.44464625e-03 -5.37374988e-03
  -2.26396974e-02 -1.75303202e-02  2.58368975e-03  5.13221361e-02
  -1.20372726e-02 -1.47177884e-03 -1.02155684e-02  2.06841869e-04
  -1.47220714e-03  1.18083227e-02 -1.05377836e-02  2.82025151e-02
   7.02104019e-03  1.60230063e-02  3.56358662e-03 -3.29516316e-03
   5.74108539e-03  1.28471414e-02 -2.39825584e-02  1.06805833e-02
   1.22822085e-02  1.03235254e-02  1.32977320e-02 -1.92087162e-02
  -8.32315627e-03 -2.64236387e-02  1.34317828e-02  2.79835030e-03
   1.29854400e-02  9.04430076e-03 -8.98786541e-03  6.09696796e-03
   

In [117]:
my_vector_ = np.squeeze(my_vector.to_numpy())
titanic_vector_ = np.squeeze(titanic_vector.to_numpy())
np.dot(my_vector_, titanic_vector_)

0.7998388

In [125]:
favorite_movie = 'titanic'
movie_id = movie_to_id[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=5)
similar_movie

(array([1626, 3499, 3569, 2596, 2973], dtype=int32),
 array([1.        , 0.24825476, 0.18911941, 0.15748177, 0.1419103 ],
       dtype=float32))

In [127]:
len(similar_movie[0])

5

In [129]:
# favorite_movie는 무시하고
# 리스트로 저장합니다.
# 하지만 연관성이 그리 높아 보이지는 않네요
# array([0.24825476, 0.18911941, 0.15748177, 0.1419103 ]
ls_recem_movie = [id_to_movie[i] for i in similar_movie[0]][1:]
ls_recem_movie

['simon sez',
 'gay deceivers, the',
 'kama sutra: a tale of love',
 'associate, the']

In [130]:
favorite_movie = 'godfather, the'
movie_id = movie_to_id[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=5)
similar_movie

(array([ 607,  380, 3150, 3531,   44], dtype=int32),
 array([0.9999999 , 0.2884657 , 0.12505478, 0.1020557 , 0.08755737],
       dtype=float32))

In [131]:
favorite_movie = 'men in black'
movie_id = movie_to_id[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=5)
similar_movie

(array([175, 107,  92,  62, 150], dtype=int32),
 array([0.9999999 , 0.15914491, 0.12767158, 0.10559031, 0.09837474],
       dtype=float32))